# Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# make sure the runtime type is GPU
import zipfile

zip_ref = zipfile.ZipFile("/content/drive/MyDrive/CustomDataset.zip", 'r')
zip_ref.extractall("/content/dataset")
zip_ref.close()

In [ ]:
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_dataset(data_dir='/content/dataset/CustomDataset', image_size=(32, 32), batch_size=32):
    # Create an instance of ImageDataGenerator with pixel values scaled to [0, 1]
    datagen = ImageDataGenerator(rescale=1/255)

    train_generator = datagen.flow_from_directory(
        os.path.join(data_dir, 'train'),
        # Resize all images to this size
        target_size=image_size,
        batch_size=batch_size,
        # Use one-hot encoded labels
        class_mode='categorical'
    )

    test_generator = datagen.flow_from_directory(
        os.path.join(data_dir, 'test'),
        # Resize all images to this size
        target_size=image_size,
        batch_size=batch_size,
        # Use one-hot encoded labels
        class_mode='categorical'
    )

    # Fetch the first batch of train data (images and labels) from the generator
    train_images, train_labels = next(train_generator)
    # Fetch the first batch of test data (images and labels) from the generator
    test_images, test_labels = next(test_generator)

    # Keep fetching batches of train data until all samples are loaded
    while len(train_images) < train_generator.samples:
        batch_images, batch_labels = next(train_generator)
        # Concatenate the new batch with the already loaded test data
        train_images = np.concatenate([train_images, batch_images], axis=0)
        train_labels = np.concatenate([train_labels, batch_labels], axis=0)

    # Keep fetching batches of test data until all samples are loaded
    while len(test_images) < test_generator.samples:
        batch_images, batch_labels = next(test_generator)
        # Concatenate the new batch with the already loaded test data
        test_images = np.concatenate([test_images, batch_images], axis=0)
        test_labels = np.concatenate([test_labels, batch_labels], axis=0)

    return train_images, train_labels, test_images, test_labels


train_images, train_labels, test_images, test_labels = load_dataset()

print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)

Found 8804 images belonging to 3 classes.
Found 3775 images belonging to 3 classes.
Train images shape: (8804, 32, 32, 3)
Train labels shape: (8804, 3)
Test images shape: (3775, 32, 32, 3)
Test labels shape: (3775, 3)


# Define Model Preparation

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D(2, 2))
  # Dropout: avoid overfiting
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D(2, 2))
  model.add(Dropout(0.2))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D(2, 2))
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dropout(0.2))
  # In this case, number of class is 3
  model.add(Dense(3, activation='softmax'))
  # compile model
  opt = SGD(learning_rate=0.001, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

model = define_model()

# Train Model

In [ ]:
history = model.fit(train_images, train_labels, epochs=100, batch_size=32, validation_data=(test_images, test_labels), verbose = 1, shuffle=True)

Epoch 1/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.5295 - loss: 0.9874 - val_accuracy: 0.7060 - val_loss: 0.6027
Epoch 2/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7548 - loss: 0.5562 - val_accuracy: 0.7764 - val_loss: 0.4862
Epoch 3/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8037 - loss: 0.4614 - val_accuracy: 0.8763 - val_loss: 0.3440
Epoch 4/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8279 - loss: 0.4078 - val_accuracy: 0.8318 - val_loss: 0.3875
Epoch 5/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8622 - loss: 0.3441 - val_accuracy: 0.8842 - val_loss: 0.2889
Epoch 6/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8814 - loss: 0.3119 - val_accuracy: 0.8562 - val_loss: 0.3561
Epoch 7/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8865 - loss: 0.2933 - val_accuracy: 0.8283 - val_loss: 0.4142
Epoch 8/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8887 - loss: 0.2836 - val_ac

# Evaluate Model

In [ ]:
def evaluate_model():
  _, accuracy = model.evaluate(test_images, test_labels, verbose=0)
  print('> %.3f' % (accuracy * 100.0))

evaluate_model()

> 94.570


# Present Results

In [ ]:
import matplotlib.pyplot as plt
import sys

def summarize_diagnostics(history):
	# plot loss
	plt.subplot(211)
	plt.title('Cross Entropy Loss')
	plt.plot(history.history['loss'], color='blue', label='train')
	plt.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	plt.subplot(212)
	plt.title('Classification Accuracy')
	plt.plot(history.history['accuracy'], color='blue', label='train')
	plt.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	plt.savefig(filename + '_plot.png')
	plt.close()

summarize_diagnostics(history)